# This notebook focuses on the prerequisites section of Trapheus. Executing this notebook will create the following resources
* Verify an email address to be used to send email alerts
* Create a S3 bucket where the system is going to store the cloud formation templates
* Create a VPC with 2 Private Subnets in 2 availability zones. Make sure the region passed is the same as in AWS credentials
* Create an RDS instance with the given name with MySQL DB, with credentials (root/pass12345)

> Note:  Make sure to have AWS credentials configured. Refer to [Setting up AWS credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html)

In [ ]:
email_id=input("Enter your email id to verify in SES: ")
region=input("Enter the region to create the resources: ")
s3_bucket_name=input("Enter the name of the S3 bucket to be created(Proposed Name: trapheus-cfn-s3-[account-id]-[region]): ")
db_instance_name=input("Enter the name for the RDS instance to be created: ")

In [ ]:
import boto3

In [ ]:
# Verify email
ses = boto3.client('ses')
response = ses.verify_email_identity(
  EmailAddress = email_id
)

In [ ]:
#Create S3 bucket
s3 = boto3.resource('s3')
s3.create_bucket(Bucket=s3_bucket_name,CreateBucketConfiguration={
        'LocationConstraint': region
    })

In [ ]:
# Get availability zones
response = ec2Client.describe_availability_zones(
    Filters=[
        {
            'Name': 'region-name',
            'Values': [region]
        },
        {
            'Name': 'state',
            'Values': ['available']
        },
    ]
)
if len(response['AvailabilityZones']) < 2:
    raise SystemExit("Stopping the execution. We need at least 2 availability zones to create private subnets. Make sure the region is set correctly in aws credentials")
zone1 = response['AvailabilityZones'][0]['ZoneName']
zone2 = response['AvailabilityZones'][1]['ZoneName']
print(zone1, zone2)

In [ ]:
#Create VPC
ec2 = boto3.resource('ec2')
ec2Client = boto3.client('ec2')
vpc = ec2.create_vpc(CidrBlock='10.0.0.0/16')
vpc.wait_until_available()
vpc_id=vpc.id
print(vpc.id)

In [ ]:
#Create Private Subnets
subnet1 = ec2.create_subnet(CidrBlock='10.0.0.0/19', VpcId=vpc_id, AvailabilityZone=zone1)
subnet1_id=subnet1.id
print(subnet1.id)

subnet2 = ec2.create_subnet(CidrBlock='10.0.32.0/19', VpcId=vpc_id,AvailabilityZone=zone2)
print(subnet2.id)
subnet2_id=subnet2.id


In [ ]:
#Create RDS DB Subnet
conn = boto3.client('rds')

response = conn.create_db_subnet_group(
    DBSubnetGroupName='Trapheus-rdssubnetgrp',
    DBSubnetGroupDescription='rdssubnetgrp',
    SubnetIds=[subnet1_id,subnet2_id],
)


In [ ]:
# Get DB security group id
response = ec2Client.describe_security_groups(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [vpc_id,]
        },
    {
            'Name': 'group-name',
            'Values': ["default"]
        },
    ],
)
db_security_group_id=response['SecurityGroups'][0]['GroupId']
print(db_security_group_id)


In [ ]:
db_instance_name

In [ ]:
# Create RDS
response = conn.create_db_instance(
        AllocatedStorage=10,
        DBInstanceIdentifier=db_instance_name,
        DBInstanceClass="db.t2.small",
        Engine="mysql",
        MasterUsername="root",
        MasterUserPassword="pass12345",
        Port=3306,
        VpcSecurityGroupIds=[db_security_group_id],
        DBSubnetGroupName = "Trapheus-rdssubnetgrp",
        StorageEncrypted=True
    )

In [ ]:
print("verified email: {}".format(email_id))
print("S3 bucket: {} ".format(s3_bucket_name))
print("Region: {}".format(region))
print("Availability zones: {}, {}".format(zone1,zone2))
print("VPC id: {}".format(vpc.id))
print("Subnet ids: {},{}".format(subnet1.id, subnet2.id))


# Use the above information while running Trapheus install notebook